# Softmax

## low-level

In [1]:
import torch
import torch.nn.functional as F

In [2]:
torch.manual_seed(1)

In [3]:
z = torch.FloatTensor([1,2,3])
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [4]:
print('합 :', hypothesis.sum())

합 : tensor(1.)


In [28]:
z = torch.rand(3, 5, requires_grad=True)
z

tensor([[0.9371, 0.6556, 0.3138, 0.1980, 0.4162],
        [0.2843, 0.3398, 0.5239, 0.7981, 0.7718],
        [0.0112, 0.8100, 0.6397, 0.9743, 0.8300]], requires_grad=True)

In [8]:
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


각 행의 원소들의 합은 1이 되는 텐서로 변환되었다. softmax 함수의 출력값은 결국 예측값이다. 즉, 위 텐서는 3개의 샘플에 대해서 5개의 클래스 중 어떤 클래스가 정답인지를 예측한 결과이다. 

In [9]:
y = torch.randint(5, (3,)).long()
print(y)

tensor([0, 2, 1])


In [25]:
# 각 label에 대해서 one-hot encoding을 수행한다
y_one_hot = torch.zeros_like(hypothesis)    # 입력 텐서와 동일한 모양의 텐서를 만들어주는데 0으로 채워줘
y_one_hot.scatter_(1, y.unsqueeze(1), 1)    # dim=1에 대해서 수행하고 y.unsqueeze(1)이 알려주는 위치에 숫자 1을 넣자

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [24]:
# unsqueeze(1)를 하면 (3,) 크기를 가졌던 Tensor는 (3x1) Tensor가 된다
print('y:',y)
print('y.unsqueeze(1) :\n',y.unsqueeze(1))

y: tensor([0, 2, 1])
y.unsqueeze(1) :
 tensor([[0],
        [2],
        [1]])


In [23]:
print(y_one_hot)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])


In [26]:
cost = (y_one_hot*-torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


## torch.scatter_()

* `index[0][0]`은 0이므로 tensor[`index[0][0]`][0]에 src[0][0]의 값인 1이 대입된다
* `index[0][1]`은 1이므로 tensor[`index[0][1]`][1]에 src[0][1]의 값인 2이 대입된다
* `index[0][2]`은 2이므로 tensor[`index[0][2]`][2]에 src[0][2]의 값인 3이 대입된다
* `index[0][3]`은 0이므로 tensor[`index[0][3]`][3]에 src[0][3]의 값인 4이 대입된다

index 텐서는 (1,4) 크기이므로 src 텐서의 첫번째 줄 값만 Tensor에 할당된다

In [13]:
src = torch.arange(1,11).reshape((2,5))
src

tensor([[ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10]])

In [16]:
index = torch.tensor([[0,1,2,0]])
target = torch.zeros(3, 5, dtype=src.dtype)
target

tensor([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]])

In [17]:
target.scatter_(0, index, src)

tensor([[1, 0, 0, 4, 0],
        [0, 2, 0, 0, 0],
        [0, 0, 3, 0, 0]])

## PyTorch로 softmax의 cost function 구현하기

### 1. F.softmax() + torch.log() = F.log_softmax()
low-level에서는 softmax 함수의 결과에 log를 씌울때는 다음과 같이 softmax의 출력값을 log 함수의 입력으로 사용하였다

In [29]:
# Low level
torch.log(F.softmax(z, dim=1))

tensor([[-1.2127, -1.4941, -1.8359, -1.9517, -1.7335],
        [-1.8912, -1.8358, -1.6516, -1.3775, -1.4038],
        [-2.3011, -1.5023, -1.6726, -1.3380, -1.4823]], grad_fn=<LogBackward0>)

PyTorch에서는 두 함수를 결합한 F.log_softmax()라는 도구를 제공한다

In [30]:
# High level
F.log_softmax(z, dim=1)

tensor([[-1.2127, -1.4941, -1.8359, -1.9517, -1.7335],
        [-1.8912, -1.8358, -1.6516, -1.3775, -1.4038],
        [-2.3011, -1.5023, -1.6726, -1.3380, -1.4823]],
       grad_fn=<LogSoftmaxBackward0>)

### 2. F.log_softmax() + F.nll_loss() = F.cross_entropy()

In [31]:
# Low-level
(y_one_hot*-F.log_softmax(z, dim=1)).sum(dim=1).mean()

tensor(1.4555, grad_fn=<MeanBackward0>)

In [33]:
# High-level
# nll = Negative Log Likelihood
F.nll_loss(F.log_softmax(z, dim=1),y)

tensor(1.4555, grad_fn=<NllLossBackward0>)

F.cross_entropy()는 F.log_softmax()와 F.nll_loss()를 포함하고 있다

In [34]:
F.cross_entropy(z, y)

tensor(1.4555, grad_fn=<NllLossBackward0>)

<b>F.cross_entropy는 cost function에 softmax function까지 포함하고 있음을 기억하고 있어야 구현 시 혼동하지 않는다</b>